In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!unzip KneeData.zip -d /content/drive/MyDrive/KneeDataRaw

Archive:  KneeData.zip
  inflating: /content/drive/MyDrive/KneeDataRaw/MedicalExpert-I/0Normal/NormalG0 (1).png  
  inflating: /content/drive/MyDrive/KneeDataRaw/MedicalExpert-I/0Normal/NormalG0 (10).png  
  inflating: /content/drive/MyDrive/KneeDataRaw/MedicalExpert-I/0Normal/NormalG0 (100).png  
  inflating: /content/drive/MyDrive/KneeDataRaw/MedicalExpert-I/0Normal/NormalG0 (101).png  
  inflating: /content/drive/MyDrive/KneeDataRaw/MedicalExpert-I/0Normal/NormalG0 (102).png  
  inflating: /content/drive/MyDrive/KneeDataRaw/MedicalExpert-I/0Normal/NormalG0 (103).png  
  inflating: /content/drive/MyDrive/KneeDataRaw/MedicalExpert-I/0Normal/NormalG0 (104).png  
  inflating: /content/drive/MyDrive/KneeDataRaw/MedicalExpert-I/0Normal/NormalG0 (105).png  
  inflating: /content/drive/MyDrive/KneeDataRaw/MedicalExpert-I/0Normal/NormalG0 (106).png  
  inflating: /content/drive/MyDrive/KneeDataRaw/MedicalExpert-I/0Normal/NormalG0 (107).png  
  inflating: /content/drive/MyDrive/KneeDataRaw/Me

In [ ]:
cd KneeDataRaw

/content/drive/.shortcut-targets-by-id/10TmkK24nJZTviaGxOkmZkcUkfiI-GBEb/KneeDataRaw


In [ ]:
import os
import random
import shutil
from PIL import Image
from transformers import ViTForImageClassification, AutoImageProcessor
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import torch
import numpy as np


In [ ]:
# Function to split data into train and test sets
def split_data(dataset_path, classes, split_ratio=0.8):
    for class_name in classes:
        class_path = os.path.join(dataset_path, class_name)
        images = os.listdir(class_path)
        random.shuffle(images)

        split_index = int(split_ratio * len(images))
        train_images = images[:split_index]
        test_images = images[split_index:]

        # Create train and test directories if they don't exist
        train_dir = os.path.join(dataset_path, 'train', class_name)
        test_dir = os.path.join(dataset_path, 'test', class_name)
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(test_dir, exist_ok=True)

        # Move images to train and test directories
        for img in train_images:
            shutil.move(os.path.join(class_path, img), os.path.join(train_dir, img))
        for img in test_images:
            shutil.move(os.path.join(class_path, img), os.path.join(test_dir, img))

In [ ]:
dataset_path = '' # As we already CD'd into the data

# Define the classes
classes = ['0Normal', '1Doubtful', '2Mild', '3Moderate', '4Severe']

# split_data(dataset_path, classes)

In [ ]:
# 300 x 162

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(os.path.join(dataset_path, 'train'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(dataset_path, 'test'), transform=transform)

# Prepare DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Load the pre-trained Vision Transformer model
model_name = 'google/vit-base-patch16-224'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name, num_labels=len(classes), ignore_mismatched_sizes=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Define the training loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader):
            optimizer.zero_grad()
            outputs = model(pixel_values=inputs)['logits']
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Define the evaluation function
def evaluate_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            outputs = model(pixel_values=inputs)['logits']
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / len(test_loader.dataset)
    print(f"Test Loss: {test_loss/len(test_loader)}, Accuracy: {accuracy}%")

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=5)

# Evaluate the model
evaluate_model(model, test_loader, criterion)

  0%|          | 0/83 [00:46<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Load the model for testing individual images
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=len(classes), ignore_mismatched_sizes=True)
model.load_state_dict(torch.load('transformers1.pth', map_location=torch.device('cpu')))
model.to("cpu")

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define the image preprocessing function
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")  # Ensure image is in RGB mode for PT Model
    image = transform(image)  # Apply transformations
    image = image.unsqueeze(0)  # Add batch dimension
    return image

# Predict class for an individual image
def predict_image(image_path):
    image = preprocess_image(image_path).to(device)  # Move image to the same device as the model
    with torch.no_grad():
        outputs = model(pixel_values=image)['logits']
        # _, predicted = torch.max(outputs, 1)
        # predicted_class = predicted.item()
    return outputs
    # return predicted_class


In [ ]:
image_path = '/content/ModerateG3 (76).png'
predicted_class = predict_image(image_path)
# Get the 1D array from the tensor
predicted_class = predicted_class.cpu().numpy().flatten()
min_abs = abs(predicted_class.min())
predicted_class = predicted_class + min_abs
print(f'Predicted Class: {np.argmax(predicted_class)}')
print(f'Confidence: {((predicted_class.max()/sum(predicted_class))*100).round(2)}%')
# Add percentage confidence, add lowest to each tensor value

Predicted Class: 2
Confidence: 31.07%
